In [ ]:
!pip install janome


     |████████████████████████████████| 19.7MB 10.5MB/s 


In [ ]:
!pip install jaconv

  Created wheel for jaconv: filename=jaconv-0.2.4-cp36-none-any.whl size=12284 sha256=f602e4f21304f84f2b606044e6fd78ae3dbaa79392c45a7954f3e59b3f98af97
  Stored in directory: /root/.cache/pip/wheels/e1/46/f7/85a7f89bd3263423c8530dfed16083f9a142cc0fc78c81ff32
Successfully built jaconv


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd, numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *

In [ ]:
url =  "https://bunshun.jp/" 
res = requests.get(url)
res.content#HTMLのテキストが返ってくる。

b'\n<!DOCTYPE html>\n<html>\n<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">\n<meta charset="utf-8">\n<meta name="viewport" content="width=1280">\n\n<title>\xe6\x96\x87\xe6\x98\xa5\xe3\x82\xaa\xe3\x83\xb3\xe3\x83\xa9\xe3\x82\xa4\xe3\x83\xb3 | \xe4\xb8\x96\xe3\x81\xae\xe4\xb8\xad\xe3\x81\xae\xe3\x80\x8c\xe3\x81\xbb\xe3\x82\x93\xe3\x81\xa8\xe3\x81\x86\xe3\x80\x8d\xe3\x81\x8c\xe3\x82\x8f\xe3\x81\x8b\xe3\x82\x8a\xe3\x81\xbe\xe3\x81\x99</title>\n\n<meta name="description" content="\xe6\x96\x87\xe8\x97\x9d\xe6\x98\xa5\xe7\xa7\x8b\xe3\x81\x8c\xe9\x81\x8b\xe5\x96\xb6\xe3\x81\x99\xe3\x82\x8b\xe3\x83\x8b\xe3\x83\xa5\xe3\x83\xbc\xe3\x82\xb9\xe3\x82\xb5\xe3\x82\xa4\xe3\x83\x88\xe3\x80\x82\xe6\x96\x87\xe6\x98\xa5\xe3\x82\xaa\xe3\x83\xb3\xe3\x83\xa9\xe3\x82\xa4\xe3\x83\xb3\xe3\x81\xaf\xe4\xb8\x96\xe3\x81\xae\xe4\xb8\xad\xe3\x82\x92\xe9\xa9\x9a\xe3\x81\x8b\xe3\x81\x9b\xe3\x82\x8b\xe3\x82\xb9\xe3\x82\xaf\xe3\x83\xbc\xe3\x83\x97\xe3\x81\x8b\xe3\x82\x89\

In [ ]:
#res.contentの中身を解析してsoupに渡している。
soup = BeautifulSoup(res.content , "html.parser")
soup


<!DOCTYPE html>

<html>
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
<meta charset="utf-8"/>
<meta content="width=1280" name="viewport"/>
<title>文春オンライン | 世の中の「ほんとう」がわかります</title>
<meta content="文藝春秋が運営するニュースサイト。文春オンラインは世の中を驚かせるスクープから、毎日の仕事や生活に役立つ話題までお届けする情報メディアです。" name="description"/>
<meta content="browserconfig.xml" name="msapplication-config"/>
<link href="/common/images/chrome-icon.png" rel="icon" sizes="192x192"/>
<link href="/common/images/apple-touch-icon.png" rel="apple-touch-icon-precomposed"/>
<link href="/common/images/favicon.ico" rel="shortcut icon"/>
<link href="https://bunshun.jp/list/feed/rss" rel="alternate" title="文春オンライン 新着RSS" type="application/rss+xml"/>
<meta content="722817551203885" property="fb:app_id">
<meta content="ja_JP" property="og:locale">
<meta content="文春オンライン" property="og:site_name">
<meta content="website" property="og:type">
<meta content="https://bunshun.jp/" property="og:url"/>
<meta content

In [ ]:
#記事一覧を取得して、タイトルとURLを取得
element = soup.select("ul")
element[0]

<ul class="header-nav">
<li><a class="ga_tracking " data-track-action="click" data-track-category="PC Header menu" href="/category/latest">新着</a></li>
<li><a class="ga_tracking " data-track-action="click" data-track-category="PC Header menu" href="/list/ranking/realtime">ランキング</a></li>
<li class="group" data-code="category"><a class="ga_tracking" data-track-action="click" data-track-category="PC Header menu" href="#">カテゴリ</a></li>
<li class="group" data-code="feature"><a class="ga_tracking " data-category="features" data-track-action="click" data-track-category="PC Header menu" href="/category/features">特集</a></li>
<li><a class="ga_tracking " data-track-action="click" data-track-category="PC Header menu" href="/list/series">連載</a></li>
<li><a class="ga_tracking " data-track-action="click" data-track-category="PC Header menu" href="/list/magazine/shukan-bunshun">週刊文春</a></li>
<li><a class="ga_tracking " data-track-action="click" data-track-category="PC Header menu" href="/list/magazine/

In [ ]:
#element
element[3]

<ul class="list-thumb pc col-3 flex top-articles-1">
<li class="thumb-two-third scoop">
<div class="left ">
<div class="label-icon"><p>SCOOP!</p></div>
<a href="/articles/-/40973"><img alt="安倍前首相の甥がフジテレビを退社「父親の岸信夫防衛大臣のもとで働く」" class="normal" src="https://bunshun.ismcdn.jp/mwimgs/2/0/582/img_2095098eab6e4fce42e2ce3158128bf0217342.jpg" srcset="https://bunshun.ismcdn.jp/mwimgs/2/0/582/img_2095098eab6e4fce42e2ce3158128bf0217342.jpg 1x,https://bunshun.ismcdn.jp/mwimgs/2/0/-/img_2095098eab6e4fce42e2ce3158128bf0217342.jpg 2x"/></a>
</div>
<div class="right">
<h3 class="title"><a href="/articles/-/40973">安倍前首相の甥がフジテレビを退社「父親の岸信夫防衛大臣のもとで働く」</a></h3>
<span class="subtitle">安倍晋三前首相の甥にあたる岸信千世氏（29）が、フジテレビを今月末で退社することが、「週刊文春」の取材で分かった。信千世氏は、岸信夫防衛大臣の長男で、曽祖父は岸信介元首相、祖父は安倍晋太郎元外相。安倍…</span>
<div class="cat-box"><span class="cat"><span class="list-author-icon" style="background-image:url(https://bunshun.ismcdn.jp/mwimgs/9/5/60/img_9505514009c1bb6a593f672b237ad5c316704.png);"></span><a class="flex" href="/list

In [ ]:
for i in element[3]:
    print(i)
#iには\nがあることが分かる。



<li class="thumb-two-third scoop">
<div class="left ">
<div class="label-icon"><p>SCOOP!</p></div>
<a href="/articles/-/40973"><img alt="安倍前首相の甥がフジテレビを退社「父親の岸信夫防衛大臣のもとで働く」" class="normal" src="https://bunshun.ismcdn.jp/mwimgs/2/0/582/img_2095098eab6e4fce42e2ce3158128bf0217342.jpg" srcset="https://bunshun.ismcdn.jp/mwimgs/2/0/582/img_2095098eab6e4fce42e2ce3158128bf0217342.jpg 1x,https://bunshun.ismcdn.jp/mwimgs/2/0/-/img_2095098eab6e4fce42e2ce3158128bf0217342.jpg 2x"/></a>
</div>
<div class="right">
<h3 class="title"><a href="/articles/-/40973">安倍前首相の甥がフジテレビを退社「父親の岸信夫防衛大臣のもとで働く」</a></h3>
<span class="subtitle">安倍晋三前首相の甥にあたる岸信千世氏（29）が、フジテレビを今月末で退社することが、「週刊文春」の取材で分かった。信千世氏は、岸信夫防衛大臣の長男で、曽祖父は岸信介元首相、祖父は安倍晋太郎元外相。安倍…</span>
<div class="cat-box"><span class="cat"><span class="list-author-icon" style="background-image:url(https://bunshun.ismcdn.jp/mwimgs/9/5/60/img_9505514009c1bb6a593f672b237ad5c316704.png);"></span><a class="flex" href="/list/author/596891fd7765615df0010000">「週刊文春」編集部</a></sp

In [ ]:
#これでタイトルを取得できる
for i in element[3]:
    if i != "\n":
        print(i.h3.string)

安倍前首相の甥がフジテレビを退社「父親の岸信夫防衛大臣のもとで働く」
「駅近、約1,200㎡の広大な敷地」「自宅の建て替えと地域貢献を両立する土地活用」プロが出した答えとは？
告発社長を聴取　「太陽光発電不正疑惑」日本テレビHDがようやく調査開始
「性別を変えることに関しては何も」天才IT相オードリー・タンが語る、幼少期最大のジレンマ
出資者逮捕も話題に…安倍昭恵夫人の「ウズハウス」で“3万円の部屋”に泊まってみた
「まるで薬師丸ひろ子の歌の世界なんだ」韓流歴20年の松本隆が“Wの悲劇”を感じたトップ女優とは
光浦靖子「49歳になりまして」芸歴28年・もう一つの人生も回収したい
「ごめんなさい」「今さら許さへん」中1生徒の背骨を折った体罰教師は、怒り爆発で生徒が気絶しても投げ続けた
東長崎、東久留米、東村山…西武線に「東〇〇駅」が多すぎるぞ問題　“ナゾ”の答えは？
後藤真希の美貌がヤバい！ 35歳になった“永遠のアイドル”は、なぜこんなにも輝き続けるのか


In [ ]:
for i in element[3]:
    if i != "\n":
        print(i.a["href"])

/articles/-/40973
/articles/-/40010
/articles/-/40994
/articles/-/40722
/articles/-/40875
/articles/-/40755
/articles/-/40804
/articles/-/40928
/articles/-/40911
/articles/-/40902


In [ ]:
title_list = []
url_list = []
for i in element[3]:
    if i != "\n":
        title_list.append(i.h3.string)
        url_list.append(url + i.a["href"])

In [ ]:
#確認を行う。
print(title_list[0])
print(url_list[0])

安倍前首相の甥がフジテレビを退社「父親の岸信夫防衛大臣のもとで働く」
https://bunshun.jp//articles/-/40973


In [ ]:
url_2 = url_list[0]
res2 = requests.get(url_2)
soup2 = BeautifulSoup(res2.content , "html.parser")
soup2


<!DOCTYPE html>

<html>
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
<meta charset="utf-8"/>
<meta content="width=1280" name="viewport"/>
<link href="https://bunshun.jp/articles/amp/40973" rel="amphtml"/>
<title>安倍前首相の甥がフジテレビを退社「父親の岸信夫防衛大臣のもとで働く」 | 文春オンライン</title>
<meta content="安倍晋三前首相の甥にあたる岸信千世氏（29）が、フジテレビを今月末で退社することが、「週刊文春」の取材で分かった。信千世氏は、岸信夫防衛大臣の長男で、曽祖父は岸信介元首相、祖父は安倍晋太郎元外相。安倍…" name="description"/>
<meta content="browserconfig.xml" name="msapplication-config"/>
<link href="/common/images/chrome-icon.png" rel="icon" sizes="192x192">
<link href="/common/images/apple-touch-icon.png" rel="apple-touch-icon-precomposed"/>
<link href="/common/images/favicon.ico" rel="shortcut icon"/>
<link href="https://bunshun.jp/list/feed/rss" rel="alternate" title="文春オンライン 新着RSS" type="application/rss+xml"/>
<meta content="722817551203885" property="fb:app_id">
<meta content="ja_JP" property="og:locale">
<meta content="文春オンライン" property="og:site_name"

In [ ]:
element2 = soup2.select("p")

In [ ]:
for i in element2:
    print(i.string)

None
None
None
　安倍晋三前首相の甥にあたる岸信千世氏（29）が、フジテレビを今月末で退社することが、「週刊文春」の取材で分かった。
　信千世氏は、岸信夫防衛大臣の長男で、曽祖父は岸信介元首相、祖父は安倍晋太郎元外相。安倍前首相は父の実兄で、伯父にあたる。2014年に慶応大学を卒業後、フジテレビに入社。社会部で警視庁や宮内庁、気象庁などを担当してきた。
「本人は政治部希望だったのですが、総理の甥が政治部では何かと不都合だろうということで社会部に配属されたようです」（フジテレビ関係者）
　10月12日、信千代氏の退社について、フジテレビの報道局幹部が集まる会議で、社会部長から「今月いっぱいで退社して、父親の防衛大臣のもとで働く予定です」と信千世氏の退社について説明があった。
　前出・フジテレビ関係者が明かす。
「信千世氏が秘書になるということで、社内でも衝撃が走りました。ただ、あの血筋ですから『やっぱり』という方が多いですね」
　信千世氏に聞くと、こう答えた。
「10月末で退社するのは事実です。詳しくはフジテレビの広報に聞いて頂ければと思います」
　フジテレビはこう回答した。
「岸は10月末で退社する予定です。今後につきましてはお答えする立場にございません」
　岸信夫氏の事務所にも取材を申し込んだが、期日までに回答はなかった。
　10月22日（木）発売の「週刊文春」では、安倍前首相の母・洋子氏が信千世氏をどう絶賛していたのか、信千世氏の合コンでの“定番ネタ”や人物像などを詳報する。
この記事の写真（5枚）
None
※アカウントの登録や購入についてのご質問は、各サイトのお問い合わせ窓口にご連絡ください。
ツイッターをフォローして最新記事をいち早く読もう
2020年10月22日 発売
菅＜極秘命令＞「携帯の次はNHKをやれ」
NEW
NEW
NEW
SCOOP!
SCOOP!
SCOOP!
SCOOP!
SCOOP!
SCOOP!
ABJマークは、この電子書店・電子書籍配信サービスが、著作権者からコンテンツ使用許諾を得た正規版配信サービスであることを示す登録商標（登録番号6091713号）です。
